In [26]:
import sqlite3

# Connexion à la base de données (ou création de celle-ci)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

In [58]:
cursor.execute('''
CREATE VIEW IF NOT EXISTS Mouvements_Caisse AS
SELECT N_contrat AS ID, Date_payement AS Date_Mouvement, 'Adhision' AS Motif, Mnt_paye AS Montant
FROM Abonnement
UNION ALL
SELECT N_contrat AS ID, Date_payement AS Date_Mouvement, 'Consommation' AS Motif, Mnt_paye AS Montant
FROM Pay_Consommation
UNION ALL
SELECT ID_Payment AS ID, Date_Reglement AS Date_Mouvement, 'ONEP' AS Motif, Mnt_paye AS Montant
FROM ONEP_Payment
UNION ALL
SELECT ID_Maintenance AS ID, Date_operation AS Date_Mouvement, 'Maintenance' AS Motif, Mnt_ouvrier AS Montant 
FROM Maintenance
UNION ALL
SELECT ID_Achat AS ID, Date_Achat AS Date_Mouvement, 'Charge Materiel' AS Motif, -Montant_total AS Montant
FROM Produit_Acheter
''')


In [55]:
cursor.execute("DROP VIEW IF EXISTS Mouvements_Caisse")


In [27]:
# Étape 1 : Créer une table temporaire sans clé primaire
cursor.execute('''
CREATE TABLE IF NOT EXISTS Blocks_temp (
    Block_ID INTEGER,
    Index_m3 REAL,
    Date_consome DATE
);
''')

# Étape 2 : Copier les données de l'ancienne table vers la nouvelle table
cursor.execute('''
INSERT INTO Blocks_temp (Block_ID, Index_m3, Date_consome)
SELECT Block_ID, Index_m3, Date_consome FROM Blocks;
''')

# Étape 3 : Supprimer l'ancienne table
cursor.execute('DROP TABLE Blocks;')

# Étape 4 : Renommer la table temporaire
cursor.execute('ALTER TABLE Blocks_temp RENAME TO Blocks;')

# Sauvegarder les modifications
conn.commit()
conn.close()

print("La clé primaire a été supprimée avec succès.")



OperationalError: database is locked

In [25]:
# Valider et fermer la connexion
conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [ ]:

cursor.execute('''DELETE FROM ONEP_Payment
WHERE Recu = 4444;''')


In [ ]:
cursor.execute('''
INSERT INTO Produit (Nom_produit) VALUES
    ('Compteur'),
    ('Pompe'),
    ('Filtre'),
    ('Réservoir'),
    ('Tuyau irrigation'),
    ('Capteur de niveau'),
    ('Vanne à bille'),
    ('Distributeur'),
    ('Adoucisseur'),
    ('Kit de traitement'),
    ('Divers');
''')


In [ ]:
# Création de la table info_personne
cursor.execute('''
CREATE TABLE IF NOT EXISTS info_personne (
    N_contrat TEXT PRIMARY KEY,
    N_conpteur_B TEXT,
    N_conpteur_P TEXT,
    Nom TEXT NOT NULL,
    Mnt_due REAL,
    Date_Adhesion DATE,
    Adresse TEXT
)
''')

# Création de la table Abonnement
cursor.execute('''
CREATE TABLE IF NOT EXISTS Abonnement (
    N_contrat TEXT,
    N_recue TEXT,
    Mnt_paye REAL,
    Date_payement DATE,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat)
)
''')

# Création de la table Qte_Consommation
cursor.execute('''
CREATE TABLE IF NOT EXISTS Qte_Consommation (
    N_contrat TEXT,
    Date_consome DATE,
    Quantite REAL,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat)
)
''')

# Création de la table Pay_Consommation
cursor.execute('''
CREATE TABLE IF NOT EXISTS Pay_Consommation (
    N_contrat TEXT,
    N_recue TEXT,
    Mnt_paye REAL,
    Date_paye Date,
    Date_payement DATE,
    FOREIGN KEY (N_contrat) REFERENCES info_personne(N_contrat))
''')

"""# Table Mouvement_Stock 
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Mouvement_Stock (
    ID_Mouvement INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_Mouvement DATE,
    Type_Mouvement TEXT,  -- Par exemple : "entrée", "sortie", "retour"
    Quantite REAL,  -- Quantité ajoutée ou retirée
    Description TEXT  -- Détails du mouvement (par exemple : "Sortie pour installation")
)
''')"""

# Table Achat 
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit_Acheter (
    ID_Achat INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_Achat DATE,
    Quantite_achetee REAL,
    Prix_unitaire REAL,
    Montant_total REAL AS (Quantite_achetee * Prix_unitaire),
    N_recu TEXT,
    Fournisseur TEXT
)
''')

# Table Produit
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit (
    Nom_Produit TEXT PRIMARY KEY NOT NULL
)
''')

# Table Produit_Utiliser
cursor.execute('''
CREATE TABLE IF NOT EXISTS Produit_Utiliser (
    ID_utils INTEGER PRIMARY KEY AUTOINCREMENT,
    Nom_Produit TEXT REFERENCES Produit(Nom_Produit),
    Date_utils DATE,
    Quantite_utuli REAL,
    Description TEXT
)
''')

#Table ONEP_Payement
cursor.execute('''CREATE TABLE IF NOT EXISTS ONEP_Payment (
    ID_Payment INTEGER PRIMARY KEY AUTOINCREMENT, 
    Mois_consome TEXT NOT NULL,            
    Mnt_paye REAL NOT NULL,                
    Recu TEXT NOT NULL,                    
    Date_Reglement DATE NOT NULL                  
)''')

# Table ONEP_Charge
cursor.execute('''CREATE TABLE IF NOT EXISTS ONEP_Credit (
    ID_Charge INTEGER PRIMARY KEY AUTOINCREMENT,  
    Mois_consome TEXT NOT NULL,  
    Quantite_Consome REAL,
    Mnt_a_paye REAL NOT NULL,                
    Facture TEXT                             
) ''')

# Table Maintenance
cursor.execute(''' CREATE TABLE IF NOT EXISTS Maintenance (
    ID_Maintenance INTEGER PRIMARY KEY AUTOINCREMENT,
    Date_operation DATE NOT NULL,
    Description TEXT,
    Nom_ouvrier TEXT,
    Mnt_ouvrier REAL
)
''')
               
# Table Maintenance en detail
cursor.execute(''' CREATE TABLE IF NOT EXISTS Maintenance_Details (
    ID_Detail INTEGER PRIMARY KEY AUTOINCREMENT,
    ID_Maintenance INTEGER REFERENCES Maintenance(ID_Maintenance) ON DELETE CASCADE,
    Nom_produit TEXT REFERENCES Produit(Nom_Produit),
    Quantite_produit REAL NOT NULL,
    FOREIGN KEY (ID_Maintenance) REFERENCES Maintenance(ID_Maintenance)
)
''')

cursor.execute('''CREATE TABLE IF NOT EXISTS Blocks (
    Block_ID INTEGER ,
    Index_m3 REAL NOT NULL,
    Date_consome Date
);
       ''')

In [5]:
cursor.execute("DROP VIEW IF EXISTS Mouvements_Caisse")

# Valider et fermer la connexion
conn.commit()
conn.close()

In [ ]:
import sqlite3

# Connexion à la base de données (remplacez par votre connexion de base de données)
conn = sqlite3.connect('Gestion_eau.db')
cursor = conn.cursor()

# Exécuter la requête SQL
query = '''
SELECT 
    q.N_contrat,
    strftime('%Y-%m', q.Date_consome) AS Mois_Consome,
    q.Date_consome,
    q.Quantite AS Index_m3,
    COALESCE(q.Quantite - LAG(q.Quantite) OVER (PARTITION BY q.N_contrat ORDER BY q.Date_consome), q.Quantite) AS Qte_Consomme_m3,
    (COALESCE(q.Quantite - LAG(q.Quantite) OVER (PARTITION BY q.N_contrat ORDER BY q.Date_consome), q.Quantite) * 7) AS Montant_dh
FROM 
    Qte_Consommation q;

'''

cursor.execute(query)

# Récupérer les résultats
results = cursor.fetchall()

# Afficher les résultats
for row in results:
    print(row)

# Fermer la connexion
conn.close()


('000', '2024-12', '2024-12-01', 90.0, 90.0)
('100', '2024-11', '2024-11-01', 0.0, 0.0)
('100', '2024-11', '2024-11-01', 0.0, 0.0)
('11', '2024-08', '2024-08-01', 60.0, 60.0)
('111', '2024-10', '2024-10-01', 70.0, 70.0)
('111', '2024-11', '2024-11-01', 77.0, 7.0)
('111', '2024-11', '2024-11-13', 129.0, 52.0)
('111', '2024-12', '2024-12-01', 90.0, -39.0)
('123', '2024-11', '2024-11-01', 301.0, 301.0)
('123', '2024-12', '2024-12-01', 12345.0, 12044.0)
('124', '2024-11', '2024-11-01', 56.0, 56.0)
('222', '2024-11', '2024-11-07', 145.0, 145.0)
('222', '2024-12', '2024-12-03', 160.0, 15.0)
('222', '2025-01', '2025-01-02', 179.0, 19.0)
('333', '2024-10', '2024-10-01', 50.0, 50.0)
('333', '2024-11', '2024-11-02', 56.0, 6.0)
('55', '2024-09', '2024-09-01', 5.0, 5.0)
('55', '2024-10', '2024-10-01', 10.0, 5.0)
('55', '2024-11', '2024-11-01', 15.0, 5.0)
('78', '2024-08', '2024-08-01', 12.0, 12.0)
('78', '2024-09', '2024-09-02', 18.0, 6.0)
('78', '2024-10', '2024-10-03', 25.0, 7.0)
('78', '2024-11

In [13]:
import sqlite3

# Connexion à la base de données
conn = sqlite3.connect('Gestion_eau.db')

# Création d'un curseur
cursor = conn.cursor()

# Exécution de la requête pour lister les tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Récupération et affichage des noms de tables
tables = cursor.fetchall()
print("Tables dans la base de données Gestion_eau :")
for table in tables:
    print(table[0])

# Fermeture de la connexion
conn.close()


Tables dans la base de données Gestion_eau :
info_personne
Abonnement
Qte_Consommation
Pay_Consommation


In [11]:
# Enregistrer les modifications et fermer la connexion
conn.commit()
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [ ]:
Gestion_eau_app/
├── app.py                # Fichier principal de l'application Flask
├── Gestion_eau.db        # Base de données SQLite (sera créée au premier lancement)
└── templates/            # Dossier contenant les fichiers HTML pour l'interface utilisateur
    ├── index.html        # Page d'accueil avec les choix d'actions
    ├── new_abonne.html   # Formulaire pour ajouter un nouvel abonné
    ├── new_consommation.html # Formulaire pour ajouter une consommation
    └── new_abonnement.html   # Formulaire pour ajouter un paiement d'abonnement
